In [1]:
from networkit import *
import math, sys, time, subprocess, os, numpy

In [12]:
kaHipAvailable = True
kaHipPath = "/home/moritzl/Gadgets/KaHIP/deploy/kaffpa"

In [13]:
emptylist = {}

In [14]:
graphlist = {"bacteriorhodopsin-10-10", "bacteriorhodopsin-10-2.5", "bacteriorhodopsin-10-5", "bacteriorhodopsin", "br_corr_2", "bubble", "ubiquitin"}

In [15]:
klist = [2**i for i in range(2,7)]

In [16]:
imbalance = 0.3

In [17]:
def continuousPartition(G, X, tolerance, k=0, isCharged={}):
    # validate input
    n = G.numberOfNodes()
    if len(isCharged) > 0:
        assert(len(isCharged==n))
        if k > 0:
            assert(len(isCharged) <= k)
    else:
        isCharged = [False for i in range(n)]
    assert(X > 0)
    assert(tolerance >= 0)
    if (n % X > 0 or (k>0 and k*X != n)) and tolerance == 0:
        if (k > 0):
            print("Creating", k, "partitions of size", X, "from a graph with", n, " nodes and tolerance 0 is impossible.")
        else:
            print("Creating partitions of size", X, "from a graph with", n, " nodes and tolerance 0 is impossible.")
        print("Set tolerance to 1.")
        tolerance = 1
    maxNumberOfPartitions = int(n / max(X - tolerance, 1)) if (k == 0) else k
    
    def w(i):
        """
        Weight of cutting after node i.
        TODO: consider other weights beside the single cut edge
        """
        assert(i >= 0)
        assert(i < n)
        if (i == n-1):
            return 0
        else:
            return G.weight(i,i+1)
    
    # allocate cut and predecessor table
    table = [[float("inf") for j in range(maxNumberOfPartitions)] for i in range(n)]
    pred = [[-1 for j in range(maxNumberOfPartitions)] for i in range(n)]
    
    # fill table 
    for i in range(n):
        for j in range(maxNumberOfPartitions):
            if (j == 0):
                if abs(i+1-X) <= tolerance:
                    table[i][j] = w(i)
            elif (i >= (X-tolerance)):
                windowStart = max(i-(X+tolerance),0)
                
                # make sure that no two charged nodes are in the same partition
                chargeEncountered = False
                for l in reversed(range(windowStart, i)):
                    if isCharged[l]:
                        if chargeEncountered:
                            windowStart = l+1
                            break
                        else:
                            chargeEncountered = True

                
                predList = [table[l][j-1] for l in range(max(i-(X+tolerance),0), min(i,i-(X-tolerance)+1))]
                if (len(predList) > 0):
                    minPred = min(predList)
                    table[i][j] = minPred + w(i)
                    pred[i][j] = predList.index(minPred) + max(i-(X+tolerance),0)
                    
    # get result from table
    resultlist = [table[n-1][j] for j in range(maxNumberOfPartitions)]
    if len(resultlist) == 0:
        print("No result found!")
    
    # if k is given, select best solution with k subsets. If not, select best overall solution
    if (k == 0):
        bestCutValue = min(resultlist)
        k = resultlist.index(bestCutValue) + 1
    else:
        bestCutValue = table[n-1][k-1]
        print("Best cut value for given k:", bestCutValue)
        
    result = partitioning.Partition(n)
    result.setUpperBound(k)
    
    # search best path backwards
    j = k-1
    i = n-1
    c = bestCutValue
    
    while (j > 0):
        nextI = pred[i][j]
        # assign partitions to nodes
        for l in range(nextI+1, i+1):
            result[l] = j
        j -= 1
        c -=w(i)
        i = nextI
        
    # assign partitions to first nodes not covered by previous loop
    for l in range(0, nextI+1):
        result[l] = 0
        
    # check results:
    for i in range(n):
        assert(result[i] >= 0)
        assert(result[i] < k)
        
    for size in result.subsetSizes():
        if (abs(size-X) > tolerance):
            print("For n=", n, ", k=", k, ", X=", X, ", tolerance=", tolerance, ", ", size, " is wrong.")
        assert(abs(size-X) <= tolerance)
    
    return result

In [8]:
def readInDistances(path):
    # read distances into array
    f = open(path, 'r')
    dd = numpy.fromfile(f, dtype=float, count=-1, sep=' ')
    f.close()

    # first number in file is number of residues
    num_res = int(dd[0])

    dists = numpy.zeros((num_res,num_res))
    dist_to_mol2 = numpy.zeros(num_res)

    dd_line = num_res+1

    # write distances into distance matrix
    for i in range(num_res):
        dist_to_mol2[i]=dd[i+1]
        for j in range(num_res-i):
            dists[i,j+i] = dd[dd_line+j]
            dists[j+i,i] = dd[dd_line+j]
        dd_line = dd_line+num_res-i
        
    return dists, dist_to_mol2

In [9]:
def qmError(part, dists, dist_to_mol2):
    """
    Calculates the quantum mechanical error induced by this partition
    Parameters
    -----------
    part - partition
    dists - distances between any pair of vertices
    dist_to_mol2 - distance to point of interest
    """
    
    strong = 5.0
    weak = 10.0
    n = len(part)
    sum_edges = 0

    
    assert(len(dists) == n)
    assert(len(dist_to_mol2) == n)
    errorSum = 0
    for i in range(n):
        for j in range(i+1, n):
            if (j == i+1) : # these are the strong bonds between the residues along the chain
                w = strong/numpy.power((dist_to_mol2[i] + dist_to_mol2[j])/2,3) 

                if (part[i] != part[j]) :
                    sum_edges+=w

            elif (j == i) : # no edge with itself
                pass
            elif dists[i,j] < 2.5 : # these are the weak bonds between residues within a certain distance cutoff
                w = weak/numpy.power((dist_to_mol2[i] + dist_to_mol2[j])/2,3)
                if (part[i] != part[j]) :
                    sum_edges+=w
    return sum_edges

In [10]:
def getKaHiPPartition(G, k, imbalance=0.03):
    factor = 10**4#this is not accurate enough, but as of now, KaHiP does not allow scientific notation

    G2 = Graph(G.numberOfNodes(), True)

    for edge in G.edges():
        G2.addEdge(edge[0], edge[1], int(G.weight(edge[0], edge[1])*factor))

    tempfilename = "scaled.graph"
    kargument = "--k="+str(k)
    iargument = "--imbalance="+str(imbalance)
    writeGraph(G2, tempfilename, Format.METIS)
    subprocess.call([kaHipPath, tempfilename, kargument, iargument])
    temppartname = "tmppartition"+str(k)
    part = community.PartitionReader().read(temppartname)
    subprocess.call(["rm", tempfilename, temppartname])
    return part

In [18]:
sumMLP = 0
sumNaive = 0
sumKaHiP = 0
sumDynCont = 0
sumNaiveBetterThanMLP = 0
sumTimeMLP = 0
sumTimeNaive = 0
sumTimeKaHip = 0
sumTimeDyn = 0
numberOfRuns = len(klist)*len(graphlist)
#dists = numpy.zeros((num_res,num_res))
#dist_to_mol2 = numpy.zeros(num_res)

for filename in graphlist:
    G = readGraph('../../input/'+filename+'.graph', Format.METIS)
    n = G.numberOfNodes()
    total = G.totalEdgeWeight()
    
    print("Graph "+filename,", ", n, " nodes.", flush=True)
    distancesAvailable = False
    if (filename == "ubiquitin"):
        dists, dist_to_mol2 = readInDistances("ubi_dists.dat")
        distancesAvailable = True
    elif (filename == "bubble"): 
        dists, dist_to_mol2 = readInDistances("bubble_dists.dat")
        distancesAvailable = True
    
    for k in klist:
        best = -1
        bestCut = total
        print("k: "+str(k))
        
        #
        # MultiLevelPartitioner
        #
        mlp = partitioning.MultiLevelPartitioner(G, k, imbalance, False, emptylist, False)
        start = time.time()
        mlp.run()
        end = time.time()
        part = mlp.getPartition()
        mlpCut = partitioning.computeEdgeCut(part, G)
        mlpImbalance = partitioning.computeImbalance(part, G)
        print("MLP")
        print("Edge cut:"+str(mlpCut))
        print("Imbalance: "+str(mlpImbalance), flush=True)
        if (distancesAvailable):
            print("QM_Error: "+str(qmError(part, dists, dist_to_mol2)))
        if (mlpImbalance <= 1+imbalance):
            best = 0
            bestCut = mlpCut
        sumTimeMLP += end - start
        
        #
        # Naive Partitioning
        #
        start = time.time()
        naivePart = community.ClusteringGenerator().makeContinuousBalancedClustering(G, k)
        end = time.time()
        naiveCut = partitioning.computeEdgeCut(naivePart, G)
        naiveImbalance = partitioning.computeImbalance(naivePart, G)
        print("Naive")
        print("Edge cut:"+str(naiveCut))
        print("Imbalance: "+str(naiveImbalance), flush=True)
        if (distancesAvailable):
            print("QM_Error: "+str(qmError(naivePart, dists, dist_to_mol2)))
        if (naiveImbalance <= 1+imbalance and naiveCut < bestCut):
            best = 1
            bestCut = naiveCut
            sumNaiveBetterThanMLP += 1
            print("MLP worse than naive. Sad.")
        sumTimeNaive += end - start
        
        #
        # Continuous Partitioning, built with dynamic programming
        #
        start = time.time()
        cont = continuousPartition(G, round(n/k), round(imbalance*round(n/k)), k)
        end = time.time()
        dynCut = partitioning.computeEdgeCut(cont, G)
        dynImbalance = partitioning.computeImbalance(cont, G)
        print("DynCont:")
        assert(k==cont.numberOfSubsets())
        print("Edge cut:"+str(dynCut))
        print("Imbalance: "+str(dynImbalance), flush=True)
        if (distancesAvailable):
            print("QM_Error: "+str(qmError(cont, dists, dist_to_mol2)))
        if (dynImbalance <= 1+imbalance and dynCut < bestCut):
            best = 3
            bestCut = dynCut
        sumTimeDyn += end - start

        #
        # Calling KaHiP, an external partitioner
        #
        if (kaHipAvailable):
            start = time.time()
            kahipPart = getKaHiPPartition(G, k, imbalance)
            end = time.time()
            kahipCut = partitioning.computeEdgeCut(kahipPart, G)
            kahipImbalance = partitioning.computeImbalance(kahipPart, G)
            print("Kaffpa")
            print("Edge cut:"+str(kahipCut))
            print("Imbalance: "+str(kahipImbalance), flush=True)
            if (distancesAvailable):
                print("QM_Error: "+str(qmError(kahipPart, dists, dist_to_mol2)))
            if (kahipImbalance <= 1+imbalance and kahipCut < bestCut):
                best = 2
                bestCut = kahipCut
            sumTimeKaHip += end - start
        
        if (best == 0):
            sumMLP += 1
        elif (best == 1):
            sumNaive += 1
        elif (best == 2):
            sumKaHiP += 1
        elif (best == 3):
            sumDynCont += 1
        else:
            print("No algorithm returned a solution of acceptable balance.")
        
        print("------------------------------------")

print("Score, mlp:", sumMLP, " naive: ", sumNaive, ", KaHiP:", sumKaHiP, ", Cont: ", sumDynCont)
print("In ", sumNaiveBetterThanMLP, " cases, the naive solution was better than the multi-level one.")
print("Average time in seconds, mlp:", sumTimeMLP / numberOfRuns, "naive:", sumTimeNaive / numberOfRuns, "KaHiP:", sumTimeKaHip)

Graph bacteriorhodopsin-10-10 ,  227  nodes.
k: 4
MLP
Edge cut:219.39286800000002
Imbalance: 1.0
Naive
Edge cut:338.357947
Imbalance: 1.0
Best cut value for given k: 1.6100109999999996
DynCont:
Edge cut:307.82478399999997
Imbalance: 1.2280701754385965
Kaffpa
Edge cut:222.0001750000001
Imbalance: 1.0
------------------------------------
k: 8
MLP
Edge cut:378.11336000000006
Imbalance: 1.0
Naive
Edge cut:436.231509
Imbalance: 1.0
Best cut value for given k: 4.816353
DynCont:
Edge cut:427.69700000000006
Imbalance: 1.2413793103448276
Kaffpa
Edge cut:351.8153570000001
Imbalance: 1.0
------------------------------------
k: 16
MLP
Edge cut:559.8249019999997
Imbalance: 1.2
Naive
Edge cut:528.2835499999999
Imbalance: 1.0
MLP worse than naive. Sad.
Best cut value for given k: 14.306756
DynCont:
Edge cut:497.0078109999998
Imbalance: 1.2
Kaffpa
Edge cut:479.54318900000004
Imbalance: 1.0
------------------------------------
k: 32
MLP
Edge cut:662.9394099999998
Imbalance: 1.125
Naive
Edge cut:655.807